In [ ]:
# In this session, we analyse the effect of the variable "Offer" on the
# "Total Sale Amount" and the "Number of Products Sold".

# Check the project workspace.
import os
print("Project Workspace:", os.getcwd())
print()

# Import all necessary packages for the project.
import pandas as pan
import glob as glob
import warnings
warnings.filterwarnings("ignore")
import statistics as myStats
from math import floor
import seaborn as sb
import matplotlib.pyplot as plt
plt.style.use("seaborn-notebook")
from statsmodels.stats.multicomp import MultiComparison

# Revome scientific notation to expand numbers in the dataset up to the
# second decimal place.
pan.set_option("display.float_format", lambda x: "%.4f" % x)
pan.set_option("display.max_rows", 1000000000)
pan.set_option("display.max_columns", 1000000000)

In [ ]:
# From the "Sale Date" data, we get the day, month and date columns for each transaction.
def fetchMonthAndYearColumns(dataFile):
    dataFile["Day"] = dataFile["Sale_Date"].apply(lambda month: month.split("-")[2])
    dataFile["Month"] = dataFile["Sale_Date"].apply(lambda month: month.split("-")[1])
    dataFile["Year"] = dataFile["Sale_Date"].apply(lambda year: year.split("-")[0])
    dataFile.drop(columns = ["Sale_Date"])
    return dataFile

# Import data for each day and collate the data. For this, we create a function
# "importAndCollate()".
def importAndCollate():
    dataFileList = []
    yearList = ['<insert year here>']
    for year in yearList:
        fileNames = [fileName for fileName in glob.glob(year + "/*.csv", recursive = True)]
        for fileName in fileNames:
            dataFileList.append(pan.read_csv(fileName))
    yearDataFile = pan.concat(dataFileList, ignore_index = False)
    print("Size of Dataset:", yearDataFile.shape)
    print()
    return yearDataFile

In [ ]:
# Import all data for a year and collate it in one dataframe.
yearDataFile = importAndCollate()

In [ ]:
# Transform the "Sale Date" column into three different columns, one each
# for "Sale Day", "Sale Month" and "Sale Year"
yearDataFile = fetchMonthAndYearColumns(yearDataFile)
yearDataFile.head(10)

In [ ]:
# Let's explore the data for the month of January (Month == "01").
januaryData = yearDataFile[yearDataFile["Month"] == "01"]
januaryDataOffer = januaryData[["Day", "Month", "Year", "UniSA_Receipt_No1", "UniSA_Customer_No", "Item_Description", "Commodity_Name", "Quantity_Sold", "Item_Value", "Offer"]][januaryData["Offer"] == "YES"]
januaryOfferProducts = januaryDataOffer["Item_Description"].unique().tolist()
januaryDataNoOffer = januaryData[["Day", "Month", "Year", "UniSA_Receipt_No1", "UniSA_Customer_No", "Item_Description", "Commodity_Name", "Quantity_Sold", "Item_Value", "Offer"]][(januaryData["Offer"] == "NO") & (januaryData["Item_Description"].isin(januaryOfferProducts))]

In [ ]:
# Check all the different "Commodity Names" available.
januaryDataOffer["Commodity_Name"].unique().tolist()

In [ ]:
# Look for any item under any commodity name
januaryDataOffer["Item_Description"][januaryDataOffer["Commodity_Name"] == "ASIAN SECTION(DIRECT)"].value_counts()

In [ ]:
# Let's now consider one product from both datasets for January, and look
# at how offers affected purchases.
januaryDataNoOfferProd001 = januaryDataNoOffer[januaryDataNoOffer["Item_Description"] == "MAGGI CUP NDL CHICKEN     60GM"]
januaryDataNoOfferProd001["Item_Value"] = (januaryDataNoOfferProd001["Item_Value"] / januaryDataNoOfferProd001["Quantity_Sold"]).round(2)
maxPrice = januaryDataNoOfferProd001["Item_Value"].max()
januaryDataNoOfferProd001["Percentage_Discount"] = (maxPrice - januaryDataNoOfferProd001["Item_Value"] ) * 100 / maxPrice
januaryDataNoOfferProd001 = januaryDataNoOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
januaryDataNoOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(januaryDataNoOfferProd001["Quantity_Sold"][januaryDataNoOfferProd001["Percentage_Discount"] == 0.0]))
januaryDataNoOfferProd001["Sales_Volume_Difference"] = januaryDataNoOfferProd001["Quantity_Sold"] - januaryDataNoOfferProd001["Expected_Sales_Volume"]
# januaryDataNoOfferProd001 = januaryDataNoOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(januaryDataNoOfferProd001)
print() 

januaryDataOfferProd001 = januaryDataOffer[januaryDataOffer["Item_Description"] == "MAGGI CUP NDL CHICKEN     60GM"]
januaryDataOfferProd001["Item_Value"] = (januaryDataOfferProd001["Item_Value"] / januaryDataOfferProd001["Quantity_Sold"]).round(2)
januaryDataOfferProd001["Percentage_Discount"] = (maxPrice - januaryDataOfferProd001["Item_Value"]) * 100 / maxPrice
januaryDataOfferProd001 = januaryDataOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
januaryDataOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(januaryDataNoOfferProd001["Quantity_Sold"][januaryDataNoOfferProd001["Percentage_Discount"] == 0.0]))
januaryDataOfferProd001["Sales_Volume_Difference"] = januaryDataOfferProd001["Quantity_Sold"]- januaryDataOfferProd001["Expected_Sales_Volume"]
# januaryDataOfferProd001 = januaryDataOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(januaryDataOfferProd001)
print()

In [ ]:
# Let's now consider one product from both datasets for January, and look
# at how offers affected purchases.
januaryDataNoOfferProd001 = januaryDataNoOffer[januaryDataNoOffer["Item_Description"] == "OCEAN BLUE SALMON 100G"]
januaryDataNoOfferProd001["Item_Value"] = (januaryDataNoOfferProd001["Item_Value"] / januaryDataNoOfferProd001["Quantity_Sold"]).round(2)
maxPrice = januaryDataNoOfferProd001["Item_Value"].max()
januaryDataNoOfferProd001["Percentage_Discount"] = (maxPrice - januaryDataNoOfferProd001["Item_Value"] ) * 100 / maxPrice
januaryDataNoOfferProd001 = januaryDataNoOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
januaryDataNoOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(januaryDataNoOfferProd001["Quantity_Sold"][januaryDataNoOfferProd001["Percentage_Discount"] == 0.0]))
januaryDataNoOfferProd001["Sales_Volume_Difference"] = januaryDataNoOfferProd001["Quantity_Sold"] - januaryDataNoOfferProd001["Expected_Sales_Volume"]
# januaryDataNoOfferProd001 = januaryDataNoOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(januaryDataNoOfferProd001)
print()

januaryDataOfferProd001 = januaryDataOffer[januaryDataOffer["Item_Description"] == "OCEAN BLUE SALMON 100G"]
januaryDataOfferProd001["Item_Value"] = (januaryDataOfferProd001["Item_Value"] / januaryDataOfferProd001["Quantity_Sold"]).round(2)
januaryDataOfferProd001["Percentage_Discount"] = (maxPrice - januaryDataOfferProd001["Item_Value"]) * 100 / maxPrice
januaryDataOfferProd001 = januaryDataOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
januaryDataOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(januaryDataNoOfferProd001["Quantity_Sold"][januaryDataNoOfferProd001["Percentage_Discount"] == 0.0]))
januaryDataOfferProd001["Sales_Volume_Difference"] = januaryDataOfferProd001["Quantity_Sold"]- januaryDataOfferProd001["Expected_Sales_Volume"]
# januaryDataOfferProd001 = januaryDataOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(januaryDataOfferProd001)
print()

In [ ]:
# Let's now consider one product from both datasets for January, and look
# at how offers affected purchases.
januaryDataNoOfferProd001 = januaryDataNoOffer[januaryDataNoOffer["Item_Description"] == "CRYSTALSPRING SPRNG WATR  #10L"]
januaryDataNoOfferProd001["Item_Value"] = (januaryDataNoOfferProd001["Item_Value"] / januaryDataNoOfferProd001["Quantity_Sold"]).round(2)
maxPrice = januaryDataNoOfferProd001["Item_Value"].max()
januaryDataNoOfferProd001["Percentage_Discount"] = (maxPrice - januaryDataNoOfferProd001["Item_Value"] ) * 100 / maxPrice
januaryDataNoOfferProd001 = januaryDataNoOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
januaryDataNoOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(januaryDataNoOfferProd001["Quantity_Sold"][januaryDataNoOfferProd001["Percentage_Discount"] == 0.0]))
januaryDataNoOfferProd001["Sales_Volume_Difference"] = januaryDataNoOfferProd001["Quantity_Sold"] - januaryDataNoOfferProd001["Expected_Sales_Volume"]
# januaryDataNoOfferProd001 = januaryDataNoOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(januaryDataNoOfferProd001)
print()

januaryDataOfferProd001 = januaryDataOffer[januaryDataOffer["Item_Description"] == "CRYSTALSPRING SPRNG WATR  #10L"]
januaryDataOfferProd001["Item_Value"] = (januaryDataOfferProd001["Item_Value"] / januaryDataOfferProd001["Quantity_Sold"]).round(2)
januaryDataOfferProd001["Percentage_Discount"] = (maxPrice - januaryDataOfferProd001["Item_Value"]) * 100 / maxPrice
januaryDataOfferProd001 = januaryDataOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
januaryDataOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(januaryDataNoOfferProd001["Quantity_Sold"][januaryDataNoOfferProd001["Percentage_Discount"] == 0.0]))
januaryDataOfferProd001["Sales_Volume_Difference"] = januaryDataOfferProd001["Quantity_Sold"]- januaryDataOfferProd001["Expected_Sales_Volume"]
# januaryDataOfferProd001 = januaryDataOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(januaryDataOfferProd001)
print()

In [ ]:
# Let's check if there is an effect seen for the whole year as well.
yearDataOffer = yearDataFile[["Day", "Month", "Year", "UniSA_Receipt_No1", "UniSA_Customer_No", "Item_Description", "Commodity_Name", "Quantity_Sold", "Item_Value", "Offer"]][yearDataFile["Offer"] == "YES"]
yearDataOfferProducts = yearDataOffer["Item_Description"].unique().tolist()
yearDataNoOffer = yearDataFile[["Day", "Month", "Year", "UniSA_Receipt_No1", "UniSA_Customer_No", "Item_Description", "Commodity_Name", "Quantity_Sold", "Item_Value", "Offer"]][(yearDataFile["Offer"] == "NO") & (yearDataFile["Item_Description"].isin(yearDataOfferProducts))]

In [ ]:
# Let's now consider one product from both datasets for 2013, and look
# at how offers affected purchases.
yearDataNoOfferProd001 = yearDataNoOffer[yearDataNoOffer["Item_Description"] == "MAGGI CUP NDL CHICKEN     60GM"]
yearDataNoOfferProd001["Item_Value"] = (yearDataNoOfferProd001["Item_Value"] / yearDataNoOfferProd001["Quantity_Sold"]).round(2)
maxPrice = yearDataNoOfferProd001["Item_Value"].max()
yearDataNoOfferProd001["Percentage_Discount"] = round(((maxPrice - yearDataNoOfferProd001["Item_Value"] ) * 100 / maxPrice), 2)
yearDataNoOfferProd001 = yearDataNoOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
yearDataNoOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(yearDataNoOfferProd001["Quantity_Sold"][yearDataNoOfferProd001["Percentage_Discount"] == 0.0]))
yearDataNoOfferProd001["Sales_Volume_Difference"] = yearDataNoOfferProd001["Quantity_Sold"] - yearDataNoOfferProd001["Expected_Sales_Volume"]
# yearDataNoOfferProd001 = yearDataNoOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(yearDataNoOfferProd001)
mc = MultiComparison(yearDataNoOfferProd001['Sales_Volume_Difference'], yearDataNoOfferProd001['Percentage_Discount'])
mc_results = mc.tukeyhsd(alpha = 0.05)
print(mc_results)
print()

yearDataOfferProd001 = yearDataOffer[yearDataOffer["Item_Description"] == "MAGGI CUP NDL CHICKEN     60GM"]
yearDataOfferProd001["Item_Value"] = (yearDataOfferProd001["Item_Value"] / yearDataOfferProd001["Quantity_Sold"]).round(2)
yearDataOfferProd001["Percentage_Discount"] = round(((maxPrice - yearDataOfferProd001["Item_Value"]) * 100 / maxPrice), 2)
yearDataOfferProd001 = yearDataOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
yearDataOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(yearDataNoOfferProd001["Quantity_Sold"][yearDataNoOfferProd001["Percentage_Discount"] == 0.0]))
yearDataOfferProd001["Sales_Volume_Difference"] = yearDataOfferProd001["Quantity_Sold"]- yearDataOfferProd001["Expected_Sales_Volume"]
# yearDataOfferProd001 = yearDataOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
yearDataOfferProd001 = pan.concat([yearDataNoOfferProd001[yearDataNoOfferProd001["Percentage_Discount"] == 0.0], yearDataOfferProd001], axis = 0, ignore_index = True)
display(yearDataOfferProd001)
mc = MultiComparison(yearDataOfferProd001['Sales_Volume_Difference'], yearDataOfferProd001['Percentage_Discount'])
mc_results = mc.tukeyhsd(alpha = 0.05)
print(mc_results)
print()

plt.figure(figsize = (10, 8))
boxPlotDiscPercSalesVolNoOffer = sb.boxplot(x = "Percentage_Discount", y = "Sales_Volume_Difference", data = yearDataNoOfferProd001, hue = "Offer", dodge = False)
boxPlotDiscPercSalesVolNoOffer.set(xlabel = "Percentage Discount : No Promotion", ylabel = "Daily Sales Volume Difference")
boxPlotDiscPercSalesVolNoOffer.set_ylim(min(yearDataNoOfferProd001["Sales_Volume_Difference"]) - 20, max(yearDataNoOfferProd001["Sales_Volume_Difference"]) + 10)

plt.figure(figsize = (10, 8))
boxPlotDiscPercSalesVolOffer = sb.boxplot(x = "Percentage_Discount", y = "Sales_Volume_Difference", data = yearDataOfferProd001, hue = "Offer", dodge = False)
boxPlotDiscPercSalesVolOffer.set(xlabel = "Percentage Discount : Promotion", ylabel = "Daily Sales Volume Difference")
boxPlotDiscPercSalesVolOffer.set_ylim(min(yearDataOfferProd001["Sales_Volume_Difference"]) - 5, max(yearDataOfferProd001["Sales_Volume_Difference"]) + 10)

In [ ]:
# Let's now consider one product from both datasets for 2013, and look
# at how offers affected purchases.
yearDataNoOfferProd001 = yearDataNoOffer[yearDataNoOffer["Item_Description"] == "PETERS I/CRM ORIG VANILLA   2L"]
yearDataNoOfferProd001["Item_Value"] = (yearDataNoOfferProd001["Item_Value"] / yearDataNoOfferProd001["Quantity_Sold"]).round(2)
maxPrice = yearDataNoOfferProd001["Item_Value"].max()
yearDataNoOfferProd001["Percentage_Discount"] = round(((maxPrice - yearDataNoOfferProd001["Item_Value"] ) * 100 / maxPrice), 2)
yearDataNoOfferProd001 = yearDataNoOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
yearDataNoOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(yearDataNoOfferProd001["Quantity_Sold"][yearDataNoOfferProd001["Percentage_Discount"] == 0.0]))
yearDataNoOfferProd001["Sales_Volume_Difference"] = yearDataNoOfferProd001["Quantity_Sold"] - yearDataNoOfferProd001["Expected_Sales_Volume"]
# yearDataNoOfferProd001 = yearDataNoOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
display(yearDataNoOfferProd001)
mc = MultiComparison(yearDataNoOfferProd001['Sales_Volume_Difference'], yearDataNoOfferProd001['Percentage_Discount'])
mc_results = mc.tukeyhsd(alpha = 0.05)
print(mc_results)
print() 

yearDataOfferProd001 = yearDataOffer[yearDataOffer["Item_Description"] == "PETERS I/CRM ORIG VANILLA   2L"]
yearDataOfferProd001["Item_Value"] = (yearDataOfferProd001["Item_Value"] / yearDataOfferProd001["Quantity_Sold"]).round(2)
yearDataOfferProd001["Percentage_Discount"] = round(((maxPrice - yearDataOfferProd001["Item_Value"]) * 100 / maxPrice), 2)
yearDataOfferProd001 = yearDataOfferProd001.groupby(["Day", "Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "sum"})
yearDataOfferProd001["Expected_Sales_Volume"] = floor(myStats.median(yearDataNoOfferProd001["Quantity_Sold"][yearDataNoOfferProd001["Percentage_Discount"] == 0.0]))
yearDataOfferProd001["Sales_Volume_Difference"] = yearDataOfferProd001["Quantity_Sold"]- yearDataOfferProd001["Expected_Sales_Volume"]
# yearDataOfferProd001 = yearDataOfferProd001.groupby(["Month", "Year", "Item_Value", "Offer", "Percentage_Discount"], as_index = False).agg({"Quantity_Sold": "mean"}).rename(columns = {"Quantity_Sold": "Daily_Average_Sold"})
yearDataOfferProd001 = pan.concat([yearDataNoOfferProd001[yearDataNoOfferProd001["Percentage_Discount"] == 0.0], yearDataOfferProd001], axis = 0, ignore_index = True)
display(yearDataOfferProd001)
mc = MultiComparison(yearDataOfferProd001['Sales_Volume_Difference'], yearDataOfferProd001['Percentage_Discount'])
mc_results = mc.tukeyhsd(alpha = 0.05)
print(mc_results)
print()

plt.figure(figsize = (10, 8))
boxPlotDiscPercSalesVolNoOffer = sb.boxplot(x = "Percentage_Discount", y = "Sales_Volume_Difference", data = yearDataNoOfferProd001, hue = "Offer", dodge = False)
boxPlotDiscPercSalesVolNoOffer.set(xlabel = "Percentage_Discount No Offer", ylabel = "Daily Sales Volume Difference")
boxPlotDiscPercSalesVolNoOffer.set_ylim(min(yearDataNoOfferProd001["Sales_Volume_Difference"]) - 20, max(yearDataNoOfferProd001["Sales_Volume_Difference"]) + 10)
plt.figure(figsize = (10, 8))
boxPlotDiscPercSalesVolOffer = sb.boxplot(x = "Percentage_Discount", y = "Sales_Volume_Difference", data = yearDataOfferProd001, hue = "Offer", dodge = False)
boxPlotDiscPercSalesVolOffer.set(xlabel = "Percentage_Discount Offer", ylabel = "Daily Sales Volume Difference")
boxPlotDiscPercSalesVolOffer.set_ylim(min(yearDataOfferProd001["Sales_Volume_Difference"]) - 5, max(yearDataOfferProd001["Sales_Volume_Difference"]) + 10)